In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

25145


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
25140,2021-06-29,47,沖縄県,84,20716,0,181
25141,2021-06-30,47,沖縄県,67,20783,2,183
25142,2021-07-01,47,沖縄県,63,20846,4,187
25143,2021-07-02,47,沖縄県,61,20907,4,191
25144,2021-07-03,47,沖縄県,76,20983,0,191


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11770,9388,151
43,香川県,19795,2098,31
44,高知県,20865,1844,26
45,鳥取県,16585,488,2
46,鹿児島県,24610,3695,38


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
25138,2021-06-27,47,沖縄県,61,20607,0,179,506.0
25139,2021-06-28,47,沖縄県,25,20632,2,181,498.0
25140,2021-06-29,47,沖縄県,84,20716,0,181,484.0
25141,2021-06-30,47,沖縄県,67,20783,2,183,455.0
25142,2021-07-01,47,沖縄県,63,20846,4,187,456.0
25143,2021-07-02,47,沖縄県,61,20907,4,191,435.0
25144,2021-07-03,47,沖縄県,76,20983,0,191,437.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
25135,2021-06-24,47,沖縄県,62,20390,4,175,533.0,-0.354167,0.111354,-0.360825
25136,2021-06-25,47,沖縄県,82,20472,4,179,529.0,0.322581,0.173637,-0.046512
25137,2021-06-26,47,沖縄県,74,20546,0,179,506.0,-0.097561,0.141427,-0.237113
25138,2021-06-27,47,沖縄県,61,20607,0,179,506.0,-0.175676,0.169350,0.000000
25139,2021-06-28,47,沖縄県,25,20632,2,181,498.0,-0.590164,0.150615,-0.242424
25140,2021-06-29,47,沖縄県,84,20716,0,181,484.0,2.360000,0.206372,-0.142857
25141,2021-06-30,47,沖縄県,67,20783,2,183,455.0,-0.202381,0.180376,-0.302083
25142,2021-07-01,47,沖縄県,63,20846,4,187,456.0,-0.059701,0.222443,0.016129
25143,2021-07-02,47,沖縄県,61,20907,4,191,435.0,-0.031746,0.171824,-0.256098
25144,2021-07-03,47,沖縄県,76,20983,0,191,437.0,0.245902,0.220891,0.027027


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
534,2021-07-03,北海道,30,2,211.0,0.153846,-0.012731,-0.361702
1069,2021-07-03,青森県,7,0,42.0,-0.222222,0.996825,-0.363636
1604,2021-07-03,岩手県,14,1,48.0,0.166667,NaN,6.000000
2139,2021-07-03,宮城県,12,0,71.0,-0.478261,0.257867,1.400000
2674,2021-07-03,秋田県,7,0,79.0,0.000000,-0.085400,-0.666667


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12840,5274,112,2021-07-03,7,0,31.0,0.166667,0.121429,0.400000
1,京都府,13910,16643,246,2021-07-03,19,0,109.0,0.357143,0.186319,-0.050000
2,佐賀県,21935,2565,24,2021-07-03,2,0,11.0,0.000000,NaN,1.000000
3,兵庫県,14980,41005,1308,2021-07-03,22,0,159.0,-0.312500,0.564337,-0.312500
4,北海道,535,41427,1403,2021-07-03,30,2,211.0,0.153846,-0.012731,-0.361702


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12840,5274,112,2021-07-03,7,0,31.0,0.166667,0.121429,0.400000,1813859
1,京都府,13910,16643,246,2021-07-03,19,0,109.0,0.357143,0.186319,-0.050000,2545899
2,佐賀県,21935,2565,24,2021-07-03,2,0,11.0,0.000000,NaN,1.000000,823810
3,兵庫県,14980,41005,1308,2021-07-03,22,0,159.0,-0.312500,0.564337,-0.312500,5549568
4,北海道,535,41427,1403,2021-07-03,30,2,211.0,0.153846,-0.012731,-0.361702,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,535,41427,1403,2021-07-03,30,2,211.0,0.153846,-0.012731,-0.361702,5267762,78.642505,786.425051,4.005496
41,青森県,1070,2518,31,2021-07-03,7,0,42.0,-0.222222,0.996825,-0.363636,1275783,19.736899,197.368988,3.292096
19,岩手県,1605,1707,47,2021-07-03,14,1,48.0,0.166667,NaN,6.000000,1235517,13.816079,138.160786,3.885013
11,宮城県,2140,9175,90,2021-07-03,12,0,71.0,-0.478261,0.257867,1.400000,2292385,40.023818,400.238180,3.097211
36,秋田県,2675,904,20,2021-07-03,7,0,79.0,0.000000,-0.085400,-0.666667,985416,9.173791,91.737906,8.016919
15,山形県,3210,2027,47,2021-07-03,5,0,7.0,1.500000,NaN,inf,1082296,18.728703,187.287027,0.646773
35,福島県,3745,4943,161,2021-07-03,16,0,103.0,-0.157895,0.295227,-0.058824,1881981,26.264877,262.648773,5.472956
38,茨城県,4280,10621,165,2021-07-03,28,0,181.0,0.217391,0.076072,-0.096774,2921436,36.355409,363.554088,6.195583
27,栃木県,4815,7109,82,2021-07-03,28,1,170.0,-0.034483,-0.000995,-0.243243,1965516,36.168619,361.686193,8.649128
37,群馬県,5350,8048,152,2021-07-03,6,0,26.0,0.200000,0.476190,0.200000,1969439,40.864429,408.644289,1.320173


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      17061.148936
std       31479.949695
min         488.000000
25%        2541.500000
50%        5039.000000
75%       11069.500000
max      175983.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6955,175983,2238,2021-07-03,716,0,3942.0,0.084848,0.079111,0.340824,13834925,127.201991,1272.019906,28.493107
1,大阪府,14445,103762,2680,2021-07-03,148,0,724.0,0.203252,0.206289,0.681818,8849635,117.250033,1172.500335,8.181128
2,神奈川県,7490,68085,952,2021-07-03,254,0,1480.0,0.104348,0.017996,0.099567,9209442,73.929561,739.295606,16.070463
3,愛知県,12305,51231,971,2021-07-03,45,0,301.0,0.022727,0.107994,-0.021739,7575530,67.626952,676.269515,3.973319
4,埼玉県,5885,46817,832,2021-07-03,116,0,721.0,-0.072000,0.049758,0.208333,7390054,63.351364,633.513639,9.756356
5,北海道,535,41427,1403,2021-07-03,30,2,211.0,0.153846,-0.012731,-0.361702,5267762,78.642505,786.425051,4.005496
6,兵庫県,14980,41005,1308,2021-07-03,22,0,159.0,-0.312500,0.564337,-0.312500,5549568,73.888634,738.886342,2.865088
7,千葉県,6420,40733,710,2021-07-03,157,2,922.0,0.053691,0.084716,0.453704,6319772,64.453275,644.532746,14.589134
8,福岡県,21400,35611,522,2021-07-03,36,1,197.0,0.500000,0.073712,-0.100000,5129841,69.419306,694.193056,3.840275
9,沖縄県,25145,20983,191,2021-07-03,76,0,437.0,0.245902,0.220891,0.027027,1481547,141.628986,1416.289865,29.496196


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      41.094048
std       29.921105
min        8.140446
25%       22.491515
50%       30.350174
75%       53.250098
max      141.628986
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,25145,20983,191,2021-07-03,76,0,437.0,0.245902,0.220891,0.027027,1481547,141.628986,1416.289865,29.496196
26,東京都,6955,175983,2238,2021-07-03,716,0,3942.0,0.084848,0.079111,0.340824,13834925,127.201991,1272.019906,28.493107
32,神奈川県,7490,68085,952,2021-07-03,254,0,1480.0,0.104348,0.017996,0.099567,9209442,73.929561,739.295606,16.070463
5,千葉県,6420,40733,710,2021-07-03,157,2,922.0,0.053691,0.084716,0.453704,6319772,64.453275,644.532746,14.589134
33,福井県,9630,1324,35,2021-07-03,9,0,84.0,-0.400000,-0.027010,-0.500000,780053,16.973206,169.732057,10.768499
7,埼玉県,5885,46817,832,2021-07-03,116,0,721.0,-0.072000,0.049758,0.208333,7390054,63.351364,633.513639,9.756356
27,栃木県,4815,7109,82,2021-07-03,28,1,170.0,-0.034483,-0.000995,-0.243243,1965516,36.168619,361.686193,8.649128
9,大阪府,14445,103762,2680,2021-07-03,148,0,724.0,0.203252,0.206289,0.681818,8849635,117.250033,1172.500335,8.181128
36,秋田県,2675,904,20,2021-07-03,7,0,79.0,0.000000,-0.085400,-0.666667,985416,9.173791,91.737906,8.016919
10,奈良県,15515,8241,131,2021-07-03,11,0,108.0,-0.352941,0.111218,-0.450000,1353837,60.871434,608.714343,7.977327


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    31.000000
mean      0.242841
std       0.311320
min      -0.085400
25%       0.061735
50%       0.121429
75%       0.279167
max       1.099206
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
29,滋賀県,13375,5567,90,2021-07-03,4,0,30.0,0.000000,1.099206,1.000000,1420948,39.178070,391.780699,2.111267
30,熊本県,23005,6488,118,2021-07-03,2,0,17.0,1.000000,1.023810,1.000000,1769880,36.657853,366.578525,0.960517
41,青森県,1070,2518,31,2021-07-03,7,0,42.0,-0.222222,0.996825,-0.363636,1275783,19.736899,197.368988,3.292096
6,和歌山県,16050,2693,49,2021-07-03,3,0,21.0,2.000000,0.607143,2.000000,954258,28.220879,282.208795,2.200663
3,兵庫県,14980,41005,1308,2021-07-03,22,0,159.0,-0.312500,0.564337,-0.312500,5549568,73.888634,738.886342,2.865088
37,群馬県,5350,8048,152,2021-07-03,6,0,26.0,0.200000,0.476190,0.200000,1969439,40.864429,408.644289,1.320173
35,福島県,3745,4943,161,2021-07-03,16,0,103.0,-0.157895,0.295227,-0.058824,1881981,26.264877,262.648773,5.472956
16,山梨県,10165,2124,21,2021-07-03,11,0,39.0,0.571429,0.293537,-0.083333,826579,25.696273,256.962734,4.718242
39,長崎県,22470,3212,69,2021-07-03,7,0,59.0,-0.125000,0.264796,-0.500000,1350769,23.779047,237.790473,4.367882
11,宮城県,2140,9175,90,2021-07-03,12,0,71.0,-0.478261,0.257867,1.400000,2292385,40.023818,400.238180,3.097211


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
36,秋田県,2675,904,20,2021-07-03,7,0,79.0,0.000000,-0.085400,-0.666667,985416,9.173791,91.737906,8.016919
33,福井県,9630,1324,35,2021-07-03,9,0,84.0,-0.400000,-0.027010,-0.500000,780053,16.973206,169.732057,10.768499
44,高知県,20865,1844,26,2021-07-03,3,0,56.0,-0.700000,-0.012796,-0.727273,709230,26.000028,260.000282,7.895887
4,北海道,535,41427,1403,2021-07-03,30,2,211.0,0.153846,-0.012731,-0.361702,5267762,78.642505,786.425051,4.005496
27,栃木県,4815,7109,82,2021-07-03,28,1,170.0,-0.034483,-0.000995,-0.243243,1965516,36.168619,361.686193,8.649128
32,神奈川県,7490,68085,952,2021-07-03,254,0,1480.0,0.104348,0.017996,0.099567,9209442,73.929561,739.295606,16.070463
42,静岡県,11770,9388,151,2021-07-03,18,0,171.0,-0.142857,0.043811,-0.500000,3708556,25.314435,253.144351,4.610959
7,埼玉県,5885,46817,832,2021-07-03,116,0,721.0,-0.072000,0.049758,0.208333,7390054,63.351364,633.513639,9.756356
34,福岡県,21400,35611,522,2021-07-03,36,1,197.0,0.500000,0.073712,-0.100000,5129841,69.419306,694.193056,3.840275
38,茨城県,4280,10621,165,2021-07-03,28,0,181.0,0.217391,0.076072,-0.096774,2921436,36.355409,363.554088,6.195583


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

#plt.style.use("ggplot")
### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   488.     22424.875  44361.75   66298.625  88235.5   110172.375
 132109.25  154046.125 175983.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([37.,  2.,  6.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 9., 22.,  4.,  9.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="山口県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
18715,2021-06-24,35,山口県,2,3119,2,76,29.0,1.000000,NaN,inf
18716,2021-06-25,35,山口県,6,3125,1,77,30.0,2.000000,0.380272,0.200000
18717,2021-06-26,35,山口県,5,3130,0,77,29.0,-0.166667,0.327891,-0.166667
18718,2021-06-27,35,山口県,2,3132,0,77,24.0,-0.600000,0.218367,-0.714286
18719,2021-06-28,35,山口県,1,3133,0,77,22.0,-0.500000,0.228571,-0.666667
18720,2021-06-29,35,山口県,0,3133,0,77,17.0,-1.000000,-0.009524,-1.000000
18721,2021-06-30,35,山口県,8,3141,0,77,24.0,inf,NaN,7.000000
18722,2021-07-01,35,山口県,4,3145,0,77,26.0,-0.500000,NaN,1.000000
18723,2021-07-02,35,山口県,7,3152,0,77,27.0,0.750000,NaN,0.166667
18724,2021-07-03,35,山口県,5,3157,1,78,27.0,-0.285714,NaN,0.000000
